In [2]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *
from sympy import Matrix, symbols, solve_linear_system
from sympy.solvers.solveset import linsolve
import time
from joblib import Parallel, delayed
x,y,z_1,z_2,z_3 = symbols('x y z_1 z_2 z_3')
x,y,z1,z2,z3 ,b= symbols("x,y,z1,z2,z3,b")
p_112, p_102, p_002, p_012 = symbols('p_112 p_102 p_002 p_012')
p_000, p_001, p_100, p_101, p_200, p_201, p_010, p_011, p_020, p_021, p_110, p_111, p_210, p_211, p_120, p_121, p_220, p_221 = symbols('p_000 p_001 p_100 p_101 p_200 p_201 p_010 p_011 p_020 p_021 p_110 p_111 p_210 p_211 p_120 p_121 p_220 p_221')

In [3]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(factors)
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False

In [4]:
#generates random rank 3 tensors
def rank_tree(size = 100):
    return (low_tensor(size) + low_tensor(size) + low_tensor(size)) 

# random tensor of some dim
def rand_tensor(size = 100000):
    return tl.tensor(np.random.randint(1, size, size=(3,2,2)))*1.0

# generates rank 1 tensors
def low_tensor(size = 100):
    a = np.random.randint(1, size, size=3) 
    b = np.random.randint(1, size, size=2)
    c = np.random.randint(1, size, size=2)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

# checks if all subtensors have nonneg rank 3
def check(t):
    t1 = tl.tensor([t[0], t[1]])
    t2 = tl.tensor([t[1], t[2]])
    t3 = tl.tensor([t[0], t[2]])
    a1 = det(Matrix(t1[0]))
    a2 = det(Matrix(t1[1]))
    a3 = det(Matrix(t2[0]))
    a4 = det(Matrix(t2[1]))
    b1 = det(Matrix(t1[:,0]))
    b2 = det(Matrix(t1[:,1]))
    b3 = det(Matrix(t2[:,0]))
    b4 = det(Matrix(t2[:,1]))
    c1 = det(Matrix(t1[:,:,0]))
    c2 = det(Matrix(t1[:,:,1]))
    c3 = det(Matrix(t2[:,:,0]))
    c4 = det(Matrix(t2[:,:,1]))

    a5 = det(Matrix(t3[0]))
    a6 = det(Matrix(t3[1]))

    b5 = det(Matrix(t3[:,0]))
    b6 = det(Matrix(t3[:,1]))

    c5 = det(Matrix(t3[:,:,0]))
    c6 = det(Matrix(t3[:,:,1]))
    return sgn([a1, a2,a3,a4,a5,a6]) or sgn([b1,b2,b3,b4,b5,b6]) or sgn([c1,c2,c3,c4,c5,c6])

# determines sign of a list
def sgn(a):
    t = 0
    ab = 0
    for a_i in a:
        t+= abs(a_i)
        ab += a_i
    return t == abs(ab)


# checks if the give 2x2x2 tensor has nonneg rank 3
def check_r3(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    return sgn([a1,a2]) or sgn([b1,b2]) or sgn([c1,c2])

# checks if given 2x2x2 has nonneg rank 2
def check_r2(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    d1 = ineq(a1,a2,b1,b2,c1,c2,ge,ge,ge)
    d2 = ineq(a1,a2,b1,b2,c1,c2,le,le,ge)
    d3 = ineq(a1,a2,b1,b2,c1,c2,le,ge,le)
    d4 = ineq(a1,a2,b1,b2,c1,c2,ge,le,le)
    supermod = d1 or d2 or d3 or d4
    return supermod


# helpers
def ineq(a1,a2,b1,b2,c1,c2,f1,f2,f3):
    t1 = f1(a1,0) and f1(a2,0)
    t2 = f2(b1,0) and f2(b2,0)
    t3 = f3(c1,0) and f3(c2,0)
    return t1 and t2 and t3
    
def ge(a1,a2):
    return a1 >= a2

def le(a1,a2):
    return a1 <= a2

# Checks if 2x2x3 contains a 2x2x2 with nonneg 4
def check_r4(t, upper = 1):
    a1 = not check_r3(tl.tensor([t[0],t[1]]))
    a2 = not check_r3(tl.tensor([t[0],t[2]]))
    a3 = not check_r3(tl.tensor([t[1],t[2]]))
    return (a1 + a2 + a3) >= upper

def r2_sub(t, upper = 1):
    a = check_r2(tl.tensor([t[0],t[1]]))
    b = check_r2(tl.tensor([t[0],t[2]]))
    c = check_r2(tl.tensor([t[2],t[1]]))
    return (bool(a)+bool(b)+bool(c)) >= upper

In [5]:
def mat_inv(t):
    tens = t.copy()
    M_1 = Matrix([Matrix(tens[0][1]).transpose(),Matrix(tens[0][0]).transpose()])
    M_2 = Matrix([Matrix(tens[1][1]).transpose(),Matrix(tens[1][0]).transpose()])
    M_3 = Matrix([Matrix(tens[2][1]).transpose(),Matrix(tens[2][0]).transpose()])
    tens[0] = M_1
    tens[1] = M_2
    tens[2] = M_3
    return tens

def mat_trans(t):
    tens = t.copy()
    tens[0] = tens[0].transpose()
    tens[1] = tens[1].transpose()
    tens[2] = tens[2].transpose()
    return tens

def rotate(t):
    return mat_inv(mat_trans(t))


def proc_tensor(t):
    tens = t.copy()
    M_b = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,1])]]).transpose()
    M_a = Matrix([[Matrix(tens[:,0,0]), Matrix(tens[:,0,1]), Matrix(tens[:,1,0])]]).transpose()
    if abs(M_a.det()) < 10:
        print("warning det too small")
    a = 1
    b = M_b.det() / M_a.det()
    M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
                [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
                [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
    R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
    M_sub = Matrix(M[0:5,0:5])
    #sol = np.array(np.array(M_sub.inv(), dtype = "float") @ np.array(Matrix(R[0:5]), dtype = "float"), dtype = "float")
    sol = la.solve(np.array(M.T@M,dtype = "float"), np.array(M.T@R, dtype = "float"))
    return np.append(np.array([b], dtype = "float"),sol)


def test(tens):
    tens = rand_tensor()
    #tens = rank_tree()
    ret = [0] *11
    c1 = proc_tensor(tens)
    if np.all(c1>=0):
        ret[1] = 1
    c2 = proc_tensor(mat_trans(tens))
    if np.all(c2>=0):
        ret[2] = 1
    c3 = proc_tensor(mat_inv(tens))
    if np.all(c3>=0):
        ret[3] = 1
    c4 = proc_tensor(rotate(tens))
    if np.all(c4>=0):
        ret[4] = 1
    total = sum(ret)
    if total >= 1:
        ret[0] = 1
    r2 = r2_sub(tens, upper = 2)
    ret[5] = r2
    #if check_r4(tens, upper = 1):
    #    ret[6] = 1
    #    if r2_sub(tens, upper = 1):
    #        ret[9] = 1
    #        if ret[0] == 1:
    #            ret[10] = 1
    if not check_r4(tens) and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    return ret

In [6]:
def test_r2(tens):
    A = Matrix(tens[0]) * Matrix(tens[1]).inv()
    res1 = Matrix(A.eigenvects()[0][2])
    res2 = Matrix(A.eigenvects()[1][2])

    B = Matrix(tens[0]).transpose() * Matrix(tens[1]).transpose().inv()
    res3 = Matrix(B.eigenvects()[0][2])
    res4 = Matrix(B.eigenvects()[1][2])
    
    T1 = np.kron(res1,res3) * 1.0
    T2 = np.kron(res2,res4) * 1.0
    assert(np.all(T1 >= 0) or np.all(T1 <= 0))
    assert(np.all(T2 >= 0) or np.all(T2 <= 0))
    T1 = Matrix(abs(T1))
    T2 = Matrix(abs(T2))

    a = symbols('a')
    M = Matrix([[T1,T2]])
    A1 = Matrix([a,a,1,1])
    A2 = Matrix([a,1,a,1])
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])

    a1 = solve(M1.det())[0]
    A1 = Matrix([a1,a1,1,1])
    M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
    M1_sub = M1[0:3,0:3]
    R = M1[0:3,3]
    d1 = False
    if abs(M1_sub.det()) > 0:
        c1 = np.array(M1_sub.inv() @ R, dtype = "float")
        d1 = np.all(c1 >=0)

    a2 = solve(M2.det())[0]
    A2 = Matrix([a2,1,a2,1])
    M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
    M2_sub = M2[0:3,0:3]
    R = M2[0:3,3]
    d2 = False
    if abs(M2_sub.det()) > 0:
        c2 = np.array(M2_sub.inv() @ R, dtype = "float")
        d2 = np.all(c2 >=0)
    return d1 or d2

def check_comb(tens):
    t = tens.copy()
    a = check_r2(tl.tensor([t[0],t[1]]))
    b = check_r2(tl.tensor([t[0],t[2]]))
    c = check_r2(tl.tensor([t[1],t[2]]))
    a1 = False
    a2 = False
    a3 = False
    if a:
        a1 = test_r2(t)
    if b:
        a2 = test_r2(tl.tensor([t[0],t[2],t[1]]))
    if c:
        a3 = test_r2(tl.tensor([t[1],t[2],t[0]]))   
    return a1 or a2 or a3

def loop_rotations(i):
    tens = rand_tensor()
    tens = rank_tree()
    ret = [0] *11
    c1 = proc_tensor(tens)
    if np.all(c1>=0):
        ret[1] = 1
    c2 = proc_tensor(mat_trans(tens))
    if np.all(c2>=0):
        ret[2] = 1
    c3 = proc_tensor(mat_inv(tens))
    if np.all(c3>=0):
        ret[3] = 1
    c4 = proc_tensor(rotate(tens))
    if np.all(c4>=0):
        ret[4] = 1
        
    total = sum(ret)
    if total >= 1:
        ret[0] = 1
    if check_comb(tens):
        ret[5] = 1
        if ret[0] == 0:
            ret[8] = 1
        ret[0] = 1
    r2 = r2_sub(tens, upper = 2)
    #ret[5] = r2
    #if check_r4(tens, upper = 1):
    #    ret[6] = 1
    #    if r2_sub(tens, upper = 1):
    #        ret[9] = 1
    #        if ret[0] == 1:
    #            ret[10] = 1
    if not check_r4(tens) and r2:
        ret[6] = 1
        if ret[0] == 0:
            ret[7] = 1
    return ret

In [47]:
te = time.time()
total = 1000
results = Parallel(n_jobs=6)(delayed(loop_rotations)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[888, 304, 323, 332, 308, 446, 558, 66, 127, 0, 0]
64.1226019859314


In [12]:
a = False
b = False
c = False
tens = rand_tensor()
while not (a):
    tens = rank_tree()
    t = tens
    a = check_r2(tl.tensor([t[0],t[1]]))
    b = check_r2(tl.tensor([t[0],t[2]]))
    c = check_r2(tl.tensor([t[1],t[2]]))
print(a,b,c)
tens

True False True


array([[[621772., 392908.],
        [531806., 344476.]],

       [[223119., 128911.],
        [161290.,  98108.]],

       [[642455., 410575.],
        [589092., 379404.]]])

In [14]:
A = Matrix(tens[0]) * Matrix(tens[1]).inv()
#a1
res1 = Matrix(A.eigenvects()[0][2])
#b1
res2 = Matrix(A.eigenvects()[1][2])

B = Matrix(tens[0]).transpose() * Matrix(tens[1]).transpose().inv()
#a2
res3 = Matrix(B.eigenvects()[0][2])
#b2
res4 = Matrix(B.eigenvects()[1][2])

T1 = np.kron(res1,res3) * 1.0
T2 = np.kron(res2,res4) * 1.0
T2

array([[-0.641911910885065],
       [-0.428669423027320],
       [-0.605362647936252],
       [-0.404261788280772]], dtype=object)

In [15]:

assert(np.all(T1 >= 0) or np.all(T1 <= 0))
assert(np.all(T2 >= 0) or np.all(T2 <= 0))
T1 = Matrix(abs(T1))
T2 = Matrix(abs(T2))
a,b = symbols('a,b')
M = Matrix([[T1,T2]])
A1 = Matrix([a*b,a,b,1])
A2 = Matrix([a,1,a,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
M1.det()

-1254.09257139102*a*b + 2603.61940851738*a + 1368.04699245417*b - 2818.07396489711

In [16]:
a1=solve(M1.det(),a)[0]
a1

0.166666666666667*(136804699245417.0*b - 281807396489711.0)/(20901542856517.0*b - 43393656808623.0)

In [26]:
A1 = Matrix([a1 *b,a1,b,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M1_sub = M1[0:3,0:3]
R = M1[0:3,3]
res = M1_sub @ R
res

Matrix([
[98182.0*b*(136804699245417.0*b - 281807396489711.0)/(20901542856517.0*b - 43393656808623.0) + 786601.08044236],
[  425818.71276242 + 98182.0*(136804699245417.0*b - 281807396489711.0)/(20901542856517.0*b - 43393656808623.0)],
[                                                                                589092.0*b + 498551.163775446]])

In [27]:
res[0] = fraction(simplify(res[0]))[0] * fraction(simplify(res[0]))[1]
res[0] = expand(res[0])
res[0]

2.80744485986332e+32*b**3 - 8.1751972760153e+32*b**2 - 2.26251861602347e+32*b + 1.48117726881641e+33

In [33]:
LC(res[0])

2.80744485986332e+32

In [34]:
solve(res[0],b)

[-1.23006972866403 + 0.e-25*I,
 2.06594220254668 - 0.e-23*I,
 2.07609826253043 + 0.e-23*I]

In [29]:
res[1] = fraction(simplify(res[1]))[0] * fraction(simplify(res[1]))[1]
res[1] = expand(res[1])
res[1]

4.66773820567665e+32*b**2 - 1.93359603303685e+33*b + 2.0024543136095e+33

In [30]:
solve(res[1],b)

[2.06637063318601, 2.07609826253165]

In [31]:
solve(res[2],b)

[-0.846304420660009]

In [32]:
A1 = Matrix([a1 *b,a1,b,1])
M1 = Matrix([[M,A1,Matrix(tens[2].reshape(4))]])
M1_sub = M1[0:3,0:3]
R = M1[0:3,3]
M1_sub@R

Matrix([
[98182.0*b*(136804699245417.0*b - 281807396489711.0)/(20901542856517.0*b - 43393656808623.0) + 786601.08044236],
[  425818.71276242 + 98182.0*(136804699245417.0*b - 281807396489711.0)/(20901542856517.0*b - 43393656808623.0)],
[                                                                                589092.0*b + 498551.163775446]])

In [ ]:
a2= solve(M2.det())[0]
A2 = Matrix([a2,1,a2,1])
M2 = Matrix([[M,A2,Matrix(tens[2].reshape(4))]])
M2_sub = M2[0:3,0:3]
R = M2[0:3,3]
M2_sub.inv() @ R
M2_sub.det()